In [21]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from torch.utils.data import DataLoader
from PIL import Image
import numpy as np

# Define your classifier model (same architecture as used during training)
class NeuralNet(nn.Module):
    def __init__(self, input_size, output_size):
        super(NeuralNet, self).__init__()
        self.input_size = input_size
        self.fc1=nn.Linear(input_size, input_size)
        self.fc2=nn.Linear(input_size, input_size//16)
        self.outputLayer=nn.Linear(input_size//16, output_size)
        self.relu=nn.ReLU()

    def forward(self, x):
        x=x.view(-1, self.input_size)
        x=self.fc1(x)
        x=self.relu(x)
        x=self.fc2(x)
        x=self.relu(x)
        x=self.outputLayer(x)
        return x

# Function to load the image, preprocess, and extract features using ResNet152
def extract_features(image_path, device='cuda' if torch.cuda.is_available() else 'cpu'):
    # Load the pre-trained ResNet152 model and remove the fully connected layers
    resnet = models.resnet152(weights='ResNet152_Weights.DEFAULT')
    feature_extractor = nn.Sequential(*list(resnet.children())[:-1]).to(device)  # Only use conv layers

    # Set the feature extractor to evaluation mode
    feature_extractor.eval()

    # Define the image transformation pipeline
    transform = transforms.Compose([
        transforms.Resize((224, 224)),   # Resize to match ResNet input size
        transforms.ToTensor(),           # Convert image to tensor
        transforms.Normalize(            # Normalize to match ImageNet dataset
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )
    ])

    # Load and preprocess the image
    img = Image.open(image_path).convert('RGB')  # Convert to RGB if not already
    img_tensor = transform(img).unsqueeze(0).to(device)  # Add batch dimension and move to device

    # Extract features from the image
    with torch.no_grad():
        features = feature_extractor(img_tensor)
        features = features.view(features.size(0), -1)  # Flatten the features (remove extra dimensions)

    return features.cpu().numpy()  # Return features as a NumPy array

# Function to classify the extracted features
def classify_image(features, classifier, device='cuda' if torch.cuda.is_available() else 'cpu'):
    # Convert features to PyTorch tensor and move to device
    features_tensor = torch.tensor(features).to(device)

    # Set the classifier to evaluation mode
    classifier.eval()

    # Get the predicted class
    with torch.no_grad():
        output = classifier(features_tensor)
        _, predicted_class = torch.max(output, 1)  # Get the index of the class with the highest score

    return predicted_class.item()

def main(image_path, classifier_path, input_size=2048, num_classes=20, device='cuda' if torch.cuda.is_available() else 'cpu'):
    # Extract features from the image
    features = extract_features(image_path, device)

    # Load the pre-trained classifier
    classifier = NeuralNet(input_size, num_classes).to(device)
    classifier.load_state_dict(torch.load(classifier_path))

    # Classify the image based on extracted features
    predicted_class = classify_image(features, classifier, device)

    print(f'Predicted class index: {predicted_class}')

if __name__ == "__main__":
    # Path to the input image
    image_path = 'images.jpg'  # Replace with the actual path to your image
    
    # Path to the saved classifier model
    classifier_path = 'model_params.pt'  # Replace with the path to your saved model
    
    # Call the main function to perform classification
    main(image_path, classifier_path)


Predicted class index: 16


/tmp/ipykernel_3527257/317685919.py:78: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  classifier.load_state_dict(torch.load(classifier_path))
